In [242]:
import numpy as np
import json

In [243]:
#import distance data
with open(r'json/distance.json') as f:
    city_distances = json.load(f)

#import input data
with open(r'json/input.json') as f:
    city_data = json.load(f)

In [244]:
#get distance function
def get_distance(start:str, end:str, data:dict):
    dis = data[start][end] 
    if (type(data[start][end]) == str):
        return 0
    else:
        return dis
    

#get closest depot
def get_depot(city:str, data:dict):
    depots = ['Goderich', 'Toronto', 'Picton']
    lowest_dist = 1E9
    closest_depot = ""

    for depot in depots:
        dist = get_distance(city, depot, data)
        
        if (dist != 0 and dist < lowest_dist):
            lowest_dist = dist
            closest_depot = depot
    return closest_depot

In [245]:
class Truck:
    def __init__(self, starting_depot) -> None:
        
        # Truck constants
        self.hourly_wage = 20
        self.overtime_wage = 30
        self.speed = 100
        self.cost_per_km = 0.68 + self.hourly_wage / self.speed
        self.cost_per_km_overtime = 0.68 + self.overtime_wage / self.speed
        
        # Truck variables
        self.home_depot = starting_depot
        self.current_city = self.home_depot
        self.rest = False
        self.distance_driven_week = 0
        self.day_cost = 0
        self.total_cost = 0
        self.packages = 5

        # Time variables
        self.current_day = "Monday"

        # Logging
        self.distance_driven_days = {
            "Monday": 0,
            "Tuesday": 0,
            "Wednesday": 0,
            "Thursday": 0,
            "Friday": 0,
        }
        self.costs_days = {
            "Monday": 0,
            "Tuesday": 0,
            "Wednesday": 0,
            "Thursday": 0,
            "Friday": 0,
        }
        self.deliveries_info = {
            "Monday": [],
            "Tuesday": [],
            "Wednesday": [],
            "Thursday": [],
            "Friday": [],
        }

    def generate_day_info(self, weekday) -> None:
        day_info = {
                "Total_distance": self.distance_driven_days[weekday],
                "Total_cost": self.costs_days[weekday],
        }
        i = 1
        for delivery in self.deliveries_info[weekday]:
            day_info[f'Delivery_{i}'] = delivery
            i += 1
        
        return day_info

    def generate_delivery_info(self, start_t, end_t, start_city, end_city, delivery_type, start_quant, end_quant, city_distances) -> dict:
        delivery_info = {
            "Start_time": start_t,
            "End_time": end_t,
            "Start_destination":start_city,
            "End_destination":end_city,
            "Distance": get_distance(start_city, end_city, city_distances),
            "Delivery_type":delivery_type,
            "Start_quantity":start_quant,
            "End_quantity": end_quant,
        }
        return delivery_info

    def generate_truck_output(self) -> dict:
        output_dict = {
            "Total_cost": self.total_cost
        }
        weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
        for weekday in weekdays:
            output_dict[weekday] = self.generate_day_info(weekday)

        return output_dict

    def calculate_cost_from_km(self, distance, overtime:bool):
        rate = self.cost_per_km 
        if overtime:
            rate = self.cost_per_km_overtime
        return distance * self.cost_per_km
    
    def update_total_cost(self):
        self.total_cost = sum(self.costs_days.values())
    
    def move_truck(self, new_city:str, city_distances:dict, city_data:dict, weekday:str, deliver:bool=True):
        # Calculate cost
        print(city_data)
        dist = get_distance(self.current_city, new_city, city_distances)
        trip_cost = self.calculate_cost_from_km(dist, False)
        print(f'trip costs {trip_cost}')
        self.costs_days[weekday] += trip_cost

        # Update truck 
        self.update_total_cost()
        old_city = self.current_city
        self.distance_driven_days[weekday] += dist
        self.current_city = new_city
        
        # Deliver packages 
        start_packages = self.packages
        if deliver:
            num_packages_req = city_data[new_city]
            if self.packages <= num_packages_req:
                city_data[new_city] -= self.packages
                self.packages = 0
            else:
                self.packages -= city_data[new_city]
                city_data[new_city] = 0
        
        # Load up on packages 
        else:
            self.packages = 5
        
        # Create delivery log
        delivery_type = "Delivery"
        if not deliver:
            delivery_type = "Re-loading"
        
        self.deliveries_info[self.current_day].append(self.generate_delivery_info(0, 0, old_city, new_city, delivery_type, start_packages, self.packages, city_distances))

        #return (self.generate_delivery_info(0, 0, old_city, new_city, delivery_type, start_packages, self.packages, city_distances))
            

    def kill_trucker(self) -> None:
        self.rest = True


In [246]:
depots = ['Goderich', 'Toronto', 'Picton']
for city in city_distances.keys():
    lowest_dist = 1E9
    closest_depot = ""
    for depot in depots:
        dist = get_distance(city, depot, city_distances)
        
        if (dist != 0 and dist < lowest_dist):
            lowest_dist = dist
            closest_depot = depot

    print(f'{city} belongs to {closest_depot}')

Tilbury belongs to Goderich
Mississauga belongs to Toronto
Cornwall belongs to Picton
London belongs to Goderich
Windsor belongs to Goderich
Niagara_Falls belongs to Toronto
Barrie belongs to Toronto
Kingston belongs to Picton
Huntsville belongs to Toronto
North_Bay belongs to Toronto
Goderich belongs to Toronto
Toronto belongs to Picton
Picton belongs to Toronto


In [247]:
truck = Truck("Toronto")
truck.move_truck("Mississauga", city_distances, city_data, "Monday", True)
truck.move_truck("Toronto", city_distances, city_data, "Monday", False)
truck.deliveries_info

{'Tilbury': 3, 'Mississauga': 4, 'Cornwall': 9, 'London': 13, 'Windsor': 5, 'Niagara_Falls': 13, 'Barrie': 18, 'Kingston': 18, 'Huntsville': 12, 'North_Bay': 5}
trip costs 24.640000000000004
{'Tilbury': 3, 'Mississauga': 0, 'Cornwall': 9, 'London': 13, 'Windsor': 5, 'Niagara_Falls': 13, 'Barrie': 18, 'Kingston': 18, 'Huntsville': 12, 'North_Bay': 5}
trip costs 24.640000000000004


{'Monday': [{'Start_time': 0,
   'End_time': 0,
   'Start_destination': 'Toronto',
   'End_destination': 'Mississauga',
   'Distance': 28,
   'Delivery_type': 'Delivery',
   'Start_quantity': 5,
   'End_quantity': 1},
  {'Start_time': 0,
   'End_time': 0,
   'Start_destination': 'Mississauga',
   'End_destination': 'Toronto',
   'Distance': 28,
   'Delivery_type': 'Re-loading',
   'Start_quantity': 1,
   'End_quantity': 5}],
 'Tuesday': [],
 'Wednesday': [],
 'Thursday': [],
 'Friday': []}

In [248]:
truck.generate_truck_output()

{'Total_cost': 49.28000000000001,
 'Monday': {'Total_distance': 56,
  'Total_cost': 49.28000000000001,
  'Delivery_1': {'Start_time': 0,
   'End_time': 0,
   'Start_destination': 'Toronto',
   'End_destination': 'Mississauga',
   'Distance': 28,
   'Delivery_type': 'Delivery',
   'Start_quantity': 5,
   'End_quantity': 1},
  'Delivery_2': {'Start_time': 0,
   'End_time': 0,
   'Start_destination': 'Mississauga',
   'End_destination': 'Toronto',
   'Distance': 28,
   'Delivery_type': 'Re-loading',
   'Start_quantity': 1,
   'End_quantity': 5}},
 'Tuesday': {'Total_distance': 0, 'Total_cost': 0},
 'Wednesday': {'Total_distance': 0, 'Total_cost': 0},
 'Thursday': {'Total_distance': 0, 'Total_cost': 0},
 'Friday': {'Total_distance': 0, 'Total_cost': 0}}

In [254]:
my_dict = {
    "a" : 1,
    "b" : 2,
}
my_dict.items()
key = my_dict.keys()
key

dict_keys(['a', 'b'])